**24.2-1<br>Run DAG-SHORTEST-PATHS on the directed fraph of Figure 24.5, using vertex $r$ as the source.**

In [1]:
from collections import deque
def topological_sort(G):
    global time # time is a global varaible, it is accumulative
    time=0
    colour={i:'white' for i in G} # initial colour of all vertices are white
    t_discover={}
    t_finish={}
    predecessor={}
    sorted_list=deque() # to store the sorted list
    for u in G:
        if colour[u]=='white':
            dfs_visit_topo(G, u, colour, t_discover,t_finish,predecessor,sorted_list) #recursion
    return sorted_list #,t_discover, t_finish

def dfs_visit_topo(G, u, colour, t_discover,t_finish,predecessor,sorted_list):
    global time
    time+=1
    t_discover[u]=time # a white vertex is discovered
    colour[u]='grey' # and coloured to grey
    for v in G[u]:
        if colour[v]=='white':
            predecessor[v]=u
            dfs_visit_topo(G, v, colour, t_discover,t_finish,predecessor,sorted_list)
    sorted_list.appendleft(u) # a vertex fully explored is appended on the left of the list
    colour[u]='black'
    time+=1
    t_finish[u]=time 
def initialize_single_source(G,s):
    # assign attribute d for each vertex for inf, except for s
    global D
    D={k:10000 for k in G}
    D[s]=0
    
    # assign attribute pi for each vertex
    # now none of the vertex has a parent
    global PI
    PI={}    
    return D, PI
def relax(u,v,w):
    # given that variable D and PI are defined
    if D[v]>D[u]+w[u][v]:
        D[v]=D[u]+w[u][v]
        PI[v]=u 
def dag_sssp(G,w,s):
    """sssp in a dag in linear time
    G: adjacency list representation of DAG, a dictionary of list
    w: weight functions of G, a nested dictionary
    s: source
    returns:
    distance from s to every vertex in G
    parent of every vertex on the sssp
    """
    initialize_single_source(G,s)
    
    for u in topological_sort(G):
        for v in G[u]:
            relax(u,v,w)
    return D, PI

In [2]:
G={'r':['s','t'],
    's':['x','t'],
  't':['x','y','z'],
  'x':['y','z'],
  'y':['z'],
  'z':[]}
w={'r':{'s':5,'t':3},
   's':{'x':6,'t':2},
    't':{'x':7,'y':4,'z':2},
    'x':{'y':-1,'z':1},
    'y':{'z':-2},
      'z':{}}
dag_sssp(G,w,'r')          

({'r': 0, 's': 5, 't': 3, 'x': 10, 'y': 7, 'z': 5},
 {'s': 'r', 't': 'r', 'x': 't', 'y': 't', 'z': 't'})

**24.2-3<br>The PERT chart formulation given above is somewhat unnatural. In a more natural structure, vertices would represent jobds and edges would represent sequencing constraints; that is edge $(u,v)$ would indicate the job $u$ must be performed before job $v$. We would then assgin weights to vertices, not edges. Modify the DAG-SHORTEST-PATHS procedure so that it finds a longest path in a directed acyclic graph with weighted vertices in linear time.**

We need to make three modifications:
* $w_pert$: the weight functions of $G$ is expressed as a normal dictionary, with vertice:weight pairs. The weights are negated so that we are tracking the increasingly negative values during relaxation
* `relax_pert`: because the $v.d$ will be updated to $u.d +w(v)$ when a path from $s$ to $u$ through $v$ has shorter distance than current $u.d$
* `initialize_single_source_pert`: the initial value of the source vertex  $s$ is initialised at its own weight $w(u)$, instead of $0$ This is because even there is no outgoing edges, the work down from $s$ to $s$ worths $w(u)$
* `dag_sssp_pert`: takes two inputs of $G$ and $w$, firstly computes the the first vertex from topological sort as the source vertex  

In the example below, the critical path, which is a longest path through the DAG, is $20$.

In [13]:
def initialize_single_source_pert(G,s,w):
    # assign attribute d for each vertex for inf, except for s
    # s=w(s) for PERT
    global D
    D={k:10000 for k in G}
    D[s]=w[s]
    
    # assign attribute pi for each vertex
    # now none of the vertex has a parent
    global PI
    PI={}    
    return D, PI
def relax_pert(u,v,w):
    # given that variable D and PI are defined for PERT
    if D[v]>D[u]+w[v]:
        D[v]=D[u]+w[v]
        PI[v]=u
def dag_sssp_pert(G,w):
    """sssp in a dag in linear time for PERT
    G: adjacency list representation of DAG, a dictionary of list
    w: weight functions of vertices in G, a dictionary
    s: source
    returns:
    distance from s to every vertex in G
    parent of every vertex on the sssp
    """
    # we set source as the first vertex in topo. sort
    s=topological_sort(G)[0]
    initialize_single_source_pert(G,s,w)

    for u in topological_sort(G):
        for v in G[u]:
            relax_pert(u,v,w)
    return D, PI

In [15]:
G={'r':['s','t'],
    's':['x','t'],
  't':['x','y','z'],
  'x':['y','z'],
  'y':['z'],
  'z':[]}
w_pert={'r':5,
   's':6,
    't':2,
    'x':1,
    'y':2,
      'z':4}

dag_sssp_pert(G,{k:-v for k,v in w_pert.items()}) 

({'r': -5, 's': -11, 't': -13, 'x': -14, 'y': -16, 'z': -20},
 {'s': 'r', 't': 's', 'x': 't', 'y': 'x', 'z': 'y'})

**24.2-4<br>Give an efficient algorithm to count the total number of paths in a directed acyclic graph. Analyze your algorithm.**

Observe the substructure of the solution: $N_u=N_v+1$, where $N_u$ is the number of paths starting from $u$, and $v$ is the child of $u$. Thus, we can use dynamic programming that stores the number of paths of every vertice and updates it. We should check the vertices in the reverse topological order, because the number of paths of its first vertice is $0$ (it has no child, a trivial solution). The topological sort takes $O(V+E)$ time. Then we scan the adjacency list of each vertex, which requires a nested `for` loop. Because we scan each edge exactly once, the time is $O(E)$. Altogehter the total running time is $O(V+E)$; the total number of paths is the sum of number of paths of all vertices. 

For the DAG in Figure 24.5, `dag_npath` returns $44$.

In [35]:
def dag_npath(G):
    # initialise the no. paths from each vertice as 0
    npath={k:0 for k in G}
    for u in reversed(topological_sort(G)):
        
        for v in G[u]:
            npath[u]+=npath[v]+1
        
    return sum(npath.values())

In [36]:
G={'r':['s','t'],
    's':['x','t'],
  't':['x','y','z'],
  'x':['y','z'],
  'y':['z'],
  'z':[]}

dag_npath(G) 

44